In [2]:
import pandas as pd

df = pd.read_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/SKU_Bin.xlsx')
df.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df
df.to_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/SKU_BinNeu.xlsx',index=False)

In [ ]:
import pyodbc
pyodbc.drivers()

ModuleNotFoundError: No module named 'pyodbc'